In [40]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import re
import json

In [41]:
container = "TEMU6276603"
tracking = {
    "general": {
        "container_number": container
    },
    "movements": []
}
url = "https://www.shipmentlink.com/servlet/TDB1_CargoTracking.do"

In [42]:
driver = webdriver.Firefox()
driver.get(url)

In [44]:
# If necessary, click button from language popup

popup = driver.find_elements(By.ID, "shipmentlink_lang_layer")
popup_style = popup[0].value_of_css_property("display")

if popup_style != 'none':
    popup_buttons = popup[0].find_elements(By.TAG_NAME, "button")
    try:
        # print("about to click")
        popup_buttons[1].click()
        print("clicked")
        # return True
    except IndexError:
        # return False
        pass

clicked


In [45]:
# Select "search by container"

container_checkbox = driver.find_elements(By.CSS_SELECTOR, "[name=SEL][value=s_cntr]")
container_checkbox[0].click()

In [46]:
# Enter container code

container_input = driver.find_elements(By.ID, "NO")
container_input[0].send_keys("TEMU6276603")

In [ ]:
# TODO: Process captcha
# captcha_input = driver.find_elements(By.ID, "captcha_input")

In [47]:
# Press button to submit form

button = driver.find_elements(By.CSS_SELECTOR, ".f12rown3 input[type=button]")
try:
    button[1].click()
except IndexError:
    print("button not found")

In [52]:
# Estimated arrival date
elements = driver.find_elements(By.CLASS_NAME, "f13tabb2")
text_group = elements[1].text.split('\n')

try:
    tracking["general"]["estimated_arrival_date"] = text_group[1]
except IndexError:
    print("estimated arrival date not found")

In [63]:
# Vessel voyage on B/L
elements = driver.find_elements(By.CSS_SELECTOR, "td.f12wrdb2")
try:
    regex = re.match(r"(.*?) *\(.*?\)", elements[1].text)
    if regex:
        tracking["general"]["vessel_voyage"] = regex.group(1)
    else:
        print("vessel voyage regex failed")
except IndexError:
    print("vessel voyage not found")

In [66]:
elements = driver.find_elements(By.CLASS_NAME, "f12rown1")

# Container type
tracking["general"]["container_type"] = elements[2].text

# Container tara
tracking["general"]["container_tara"] = elements[8].text

In [69]:
# Movement information
elements = driver.find_elements(By.CLASS_NAME, "f12rown1")
tracking["movements"] = []
movement = {}

# Movement date
movement["date"] = elements[3].text

# Movement status
movement["status"] = elements[4].text

# Movement location
movement["location"] = elements[5].text

# Movement vessel voyage
regex = re.match(r"(.*) (.*?)$", elements[6].text)
movement["transport_ship"] = regex.group(1)
movement["transport_voyage"] = regex.group(2)

tracking["movements"].append(movement)

In [70]:
tracking

{'general': {'container_number': 'TEMU6276603',
  'container_tara': '6273 KGS',
  'container_type': "40'(SH)",
  'estimated_arrival_date': 'MAR-10-2019',
  'vessel_voyage': 'JIANG HANG 982 190222'},
 'movements': [{'date': 'MAR-04-2019',
   'location': 'SHEKOU, CHINA (CN)',
   'status': 'Transship container loaded on vessel',
   'transport_ship': 'EVER BEAMY',
   'transport_voyage': '0815-007B'}]}

In [71]:
driver.close()

In [73]:
import json
with open('evergreen/' + container + '.json', 'w') as file:
    json.dump(tracking, file)